In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv('fraudTrain.csv')
test_data = pd.read_csv('fraudTest.csv')

In [3]:
print(train_data.columns)
print(test_data.columns)

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')
Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')


In [4]:
columns_to_remove = ['Unnamed: 0','cc_num','first','last','street','city','state','zip','gender','dob','job','trans_num','unix_time','merchant']
def modifying(data,columns_to_remove):
    return data.drop(columns = columns_to_remove)

In [5]:
test_data = modifying(test_data,columns_to_remove)
print(test_data)
train_data = modifying(train_data,columns_to_remove)
print(train_data)

       trans_date_trans_time        category     amt      lat      long  \
0        2020-06-21 12:14:25   personal_care    2.86  33.9659  -80.9355   
1        2020-06-21 12:14:33   personal_care   29.84  40.3207 -110.4360   
2        2020-06-21 12:14:53  health_fitness   41.28  40.6729  -73.5365   
3        2020-06-21 12:15:15        misc_pos   60.05  28.5697  -80.8191   
4        2020-06-21 12:15:17          travel    3.19  44.2529  -85.0170   
...                      ...             ...     ...      ...       ...   
555714   2020-12-31 23:59:07  health_fitness   43.77  40.4931  -91.8912   
555715   2020-12-31 23:59:09       kids_pets  111.84  29.0393  -95.4401   
555716   2020-12-31 23:59:15       kids_pets   86.88  46.1966 -118.9017   
555717   2020-12-31 23:59:24          travel    7.99  44.6255 -116.4493   
555718   2020-12-31 23:59:34   entertainment   38.13  35.6665  -97.4798   

        city_pop  merch_lat  merch_long  is_fraud  
0         333497  33.986391  -81.200714        

In [6]:
def transformation_for_time(data):    
    data['improved_time'] = pd.to_datetime(data['trans_date_trans_time'])
    data['trans_year'] = data['improved_time'].dt.year
    data['trans_month'] = data['improved_time'].dt.month
    data['trans_day'] = data['improved_time'].dt.day
    data['trans_hour'] = data['improved_time'].dt.hour
    return data

train_data = transformation_for_time(train_data)
test_data = transformation_for_time(test_data)

In [7]:
train_data = train_data.drop(['improved_time','trans_date_trans_time'], axis = 1)
test_data = test_data.drop(['improved_time','trans_date_trans_time'], axis = 1)

In [8]:
print(test_data[:3])

         category    amt      lat      long  city_pop  merch_lat  merch_long  \
0   personal_care   2.86  33.9659  -80.9355    333497  33.986391  -81.200714   
1   personal_care  29.84  40.3207 -110.4360       302  39.450498 -109.960431   
2  health_fitness  41.28  40.6729  -73.5365     34496  40.495810  -74.196111   

   is_fraud  trans_year  trans_month  trans_day  trans_hour  
0         0        2020            6         21          12  
1         0        2020            6         21          12  
2         0        2020            6         21          12  


In [9]:
xr = train_data.drop(['is_fraud'],axis = 1)
y = train_data['is_fraud']
xe = test_data.drop(['is_fraud'],axis = 1)
ye = test_data['is_fraud']

In [10]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
preprocess = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), ['amt','lat','long','city_pop','merch_lat','merch_long','trans_year','trans_month','trans_day','trans_hour']),
        ('category', OneHotEncoder() , ['category'])
    ]
)
x_trained_prepared = preprocess.fit_transform(xr)
x_tested_prepared = preprocess.transform(xe)

In [11]:
#from imblearn.over_sampling import ADASYN
#adasyn = ADASYN(random_state=42)
#x_trained_prepared_resampled, y_resampled = adasyn.fit_resample(x_trained_prepared, y)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
model = LogisticRegression(max_iter = 1000, class_weight = "balanced")
model.fit(x_trained_prepared, y)
y_prediction = model.predict(x_tested_prepared)
classification = classification_report(ye,y_prediction)
print(classification)

              precision    recall  f1-score   support

           0       1.00      0.94      0.97    553574
           1       0.05      0.73      0.09      2145

    accuracy                           0.94    555719
   macro avg       0.52      0.84      0.53    555719
weighted avg       1.00      0.94      0.97    555719

